In [1]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
import os

def load_data():
    component = pd.read_csv("../dataset/complete_dataset.csv")
    #component = component.drop(['Project','Version','Smell'],axis=1)
    df = pd.DataFrame()

    path = "../../projects"
    projects = []

    # Itera su tutti i file e le cartelle nella cartella specificata
    for item in os.listdir(path):
        # Se l'elemento nella cartella è una cartella, aggiungi il nome alla lista
        if os.path.isdir(os.path.join(path, item)):
            projects.append(item)
    # Stampa la lista di nomi dei progetti
    print(projects)

    possible_subfolders = ["src/java", "src/main", "src/main/java", "src"]
    #classes = []

    for k in range(len(projects)):
        print(projects[k])
        for i in range(len(component)):
            line = component.loc[i,'ComponentName'].strip()
            line = line.replace(".","/") + ".java"
            for subfolder in possible_subfolders:
                full_path = os.path.join(path, projects[k], subfolder, line)
                if os.path.exists(full_path):
                    with open(full_path, "r") as f:
                        contenuto = f.read()
                    # classes.append(contenuto)
                    temp_df = pd.DataFrame({
                        'Project_name': projects[k], 
                        'Component': [contenuto],
                        'CDSBP' : [component.loc[i, 'CDSBP']],
                        'CC': [component.loc[i,'CC']],
                        'LC': [component.loc[i,'LC']],
                        'LZC': [component.loc[i,'LZC']],
                        'RB': [component.loc[i,'RB']],
                        'SC': [component.loc[i,'SC']]
                    })
                    df = pd.concat([df, temp_df], ignore_index=True)
                    break # Esci dal ciclo for se hai trovato il file
    return df

In [3]:
path = "../../projects"
projects = []
code_smells = ['CDSBP', 'CC', 'LC', 'LC', 'LZC', 'RB', 'SC']
# Itera su tutti i file e le cartelle nella cartella specificata
for item in os.listdir(path):
    # Se l'elemento nella cartella è una cartella, aggiungi il nome alla lista
    if os.path.isdir(os.path.join(path, item)):
       projects.append(item)
    # Stampa la lista di nomi dei progetti
print(projects)

['ant-ivy-2.0.0-alpha2', 'ant-rel-1.8.3', 'cassandra-cassandra-1.0.0', 'elasticsearch-v0.19.0', 'hadoop-release-0.6.0', 'hive-release-0.9.0', 'hsqldb-2.2.8', 'karaf-karaf-2.3.0', 'lucene-releases-lucene-solr-3.6.0', 'manifold-cf-release-0.6', 'nutch-release-1.4', 'pig-release-0.8.0', 'qpid-0.14', 'struts-STRUTS_2_3_4', 'xerces2-j-Xerces-J_2_3_0']


In [4]:
df = load_data()

['ant-ivy-2.0.0-alpha2', 'ant-rel-1.8.3', 'cassandra-cassandra-1.0.0', 'elasticsearch-v0.19.0', 'hadoop-release-0.6.0', 'hive-release-0.9.0', 'hsqldb-2.2.8', 'karaf-karaf-2.3.0', 'lucene-releases-lucene-solr-3.6.0', 'manifold-cf-release-0.6', 'nutch-release-1.4', 'pig-release-0.8.0', 'qpid-0.14', 'struts-STRUTS_2_3_4', 'xerces2-j-Xerces-J_2_3_0']
ant-ivy-2.0.0-alpha2
ant-rel-1.8.3
cassandra-cassandra-1.0.0
elasticsearch-v0.19.0
hadoop-release-0.6.0
hive-release-0.9.0
hsqldb-2.2.8
karaf-karaf-2.3.0
lucene-releases-lucene-solr-3.6.0
manifold-cf-release-0.6
nutch-release-1.4
pig-release-0.8.0
qpid-0.14
struts-STRUTS_2_3_4
xerces2-j-Xerces-J_2_3_0


In [5]:
df

,Project_name,Component,CDSBP,CC,LC,LZC,RB,SC
0,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
1,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
2,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
3,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
4,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
8139,xerces2-j-Xerces-J_2_3_0,"/*\n * The Apache Software License, Version 1....",0,0,0,0,0,0
8140,xerces2-j-Xerces-J_2_3_0,// XMLFilterImpl.java - base SAX2 filter imple...,0,0,0,0,0,0
8141,xerces2-j-Xerces-J_2_3_0,// XMLReaderAdapter.java - adapt an SAX2 XMLRe...,0,0,0,0,0,0
8142,xerces2-j-Xerces-J_2_3_0,// XMLReaderFactory.java - factory for creatin...,0,0,0,0,0,1


In [6]:
# Dizionario per salvare le predizioni
predictions = {}

# Dizionario per salvare le metriche
metrics = {}

# Loop attraverso i progetti
for project in projects:
    print("Current project:", project)

    # Caricamento dei dati specifici del progetto
    project_df = df[df['Project_name'] == project]

    # Inizializza il dizionario per il progetto corrente
    predictions[project] = {}
    metrics[project] = {}

    # Loop attraverso i code smell
    for smell in code_smells:
        print("Current code smell:", smell)

        X = project_df["Component"]
        y = project_df[str(smell)]

        if len(project_df) == 0:
            continue

        # Effettua lo split in train e test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

        # Definisci il classificatore
        classifier = MLPClassifier(verbose=False, hidden_layer_sizes=[10, 20, 20],activation='logistic')

        # Crea il pipeline
        pipeline = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('classifier', classifier),
        ], verbose=False)

        # Addestra il modello
        pipeline.fit(X_train, y_train)

        # Valuta il modello
        y_pred = pipeline.predict(X_test)

        # Salva le predizioni per lo smell corrente nel progetto corrente
        predictions[project][smell] = y_pred

        precision = precision_score(y_test, y_pred, average="macro")
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")
        mcc = matthews_corrcoef(y_test, y_pred)

        # Salva le metriche per lo smell corrente nel progetto corrente
        metrics[project][smell] = {
            "Precision": precision,
            "Accuracy": accuracy,
            "Recall": recall,
            "F1": f1,   # Aggiungi l'F1 score al dizionario delle metriche
            "MCC": mcc  # Aggiungi l'MCC al dizionario delle metriche
        }

        print("Accuracy:", accuracy)
        print("Recall:", recall)
        print("Precision:", precision)
        print("F1:", f1)
        print("MCC:", mcc)
        print("--------------")

Current project: ant-ivy-2.0.0-alpha2
Current code smell: CDSBP


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.963302752293578
Recall: 0.5
Precision: 0.481651376146789
F1: 0.49065420560747663
MCC: 0.0
--------------
Current code smell: CC


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1: 1.0
MCC: 0.0
--------------
Current code smell: LC


...

C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9393063583815029
Recall: 0.5
Precision: 0.46965317919075145
F1: 0.4843517138599106
MCC: 0.0
--------------
Current code smell: SC


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.9624277456647399
Recall: 0.5
Precision: 0.48121387283236994
F1: 0.49042709867452133
MCC: 0.0
--------------


C:\Users\demai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
# Stampa le predizioni per ogni progetto e smell
for project, smell_predictions in predictions.items():
    print("Project:", project)
    for smell, pred_list in smell_predictions.items():
        print("Code Smell:", smell)
        print(pred_list)
        print("--------------")


Project: ant-ivy-2.0.0-alpha2
Code Smell: CDSBP
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
--------------
Code Smell: CC
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
--------------
Code Smell: LC
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
--------------
Code Smell: LZC
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
# Stampa le metriche per ogni progetto e smell
for project, smell_metrics in metrics.items():
    print("Project:", project)
    for smell, metric_values in smell_metrics.items():
        print("Code Smell:", smell)
        print(metric_values)
        print("--------------")

In [14]:
metrics['pig-release-0.8.0']

{'CDSBP': {'Precision': 0.47104247104247104,
  'Accuracy': 0.9420849420849421,
  'Recall': 0.5,
  'F1': 0.485089463220676,
  'MCC': 0.0},
 'CC': {'Precision': 0.48455598455598453,
  'Accuracy': 0.9691119691119691,
  'Recall': 0.5,
  'F1': 0.492156862745098,
  'MCC': 0.0},
 'LC': {'Precision': 0.4980694980694981,
  'Accuracy': 0.9961389961389961,
  'Recall': 0.5,
  'F1': 0.4990328820116054,
  'MCC': 0.0},
 'LZC': {'Precision': 1.0,
  'Accuracy': 1.0,
  'Recall': 1.0,
  'F1': 1.0,
  'MCC': 0.0},
 'RB': {'Precision': 0.4864864864864865,
  'Accuracy': 0.972972972972973,
  'Recall': 0.5,
  'F1': 0.4931506849315069,
  'MCC': 0.0},
 'SC': {'Precision': 0.4749034749034749,
  'Accuracy': 0.9498069498069498,
  'Recall': 0.5,
  'F1': 0.4871287128712872,
  'MCC': 0.0}}

#### Il CountVectorizer genera una rappresentazione vettoriale in cui ogni parola unica diventa una feature e la frequenza di ciascuna parola viene conteggiata. Il TfidfTransformer, invece, calcola la frequenza termine-frequenza inversa del documento (TF-IDF) delle feature estratte dal CountVectorizer. Il TF-IDF è una misura statistica che tiene conto sia della frequenza della parola in un documento (TF) che della sua rarità nell'intero corpus (IDF). Questo aiuta a pesare le parole più significative rispetto a quelle comuni. Il TfidfTransformer moltiplica la matrice delle frequenze delle parole per il peso TF-IDF per ottenere una rappresentazione vettoriale ponderata.